In [5]:
from utils import db_setup_core
from sqlalchemy import select, and_, or_, Integer, cast, Float, func
from queries import _result_proxy_to_df

In [6]:
engine, metadata = db_setup_core()

In [7]:
def select_recording_sessions(
    engine,
    metadata,
    group_names=None,
    exp_names=None,
    session_names=None,
    exclude_excluded_recordings=True,
    as_df=True,
):
    r_sesh, groups, experiments = (
        metadata.tables["recording_sessions"],
        metadata.tables["experimental_groups"],
        metadata.tables["experiments"],
    )

    stmt = select([r_sesh, groups.c.group_name, experiments.c.experiment_name])
    stmt = stmt.select_from(r_sesh.join(groups).join(experiments))
    if exclude_excluded_recordings:
        stmt = stmt.where(or_(r_sesh.c.excluded.is_(None), r_sesh.c.excluded == 1))
    if group_names:
        stmt = stmt.where(groups.c.group_name.in_(group_names))
    if exp_names:
        stmt = stmt.where(experiments.c.experiment_name.in_(exp_names))
    if session_names:
        stmt = stmt.where(r_sesh.c.session_name.in_(session_names))

    with engine.connect() as con:
        rp = con.execute(stmt)

    if as_df:
        return _result_proxy_to_df(rp)
    else:
        return rp

In [9]:
df = select_recording_sessions(engine, metadata)


In [10]:
df.head()

,id,session_name,session_date,start_time,group_id,mouse_id,chan_map_id,excluded,group_name,experiment_name
0,1,ESHOCK_01,2019-02-28,14:09:37,2,None,1,None,shock,ESHOCK
1,2,ESHOCK_02,2019-03-05,13:39:08,2,None,1,None,shock,ESHOCK
2,3,ESHOCK_03_LOC1,2019-03-08,13:10:19,2,None,1,None,shock,ESHOCK
3,4,ESHOCK_04_LOC1,2019-03-08,14:44:40,2,None,1,None,shock,ESHOCK
4,5,ESHOCK_06_LOC1,2019-04-09,11:24:46,1,None,1,None,sham,ESHOCK


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49 entries, 0 to 48
Data columns (total 10 columns):
id                 49 non-null int64
session_name       49 non-null object
session_date       49 non-null object
start_time         49 non-null object
group_id           49 non-null int64
mouse_id           0 non-null object
chan_map_id        49 non-null int64
excluded           0 non-null object
group_name         49 non-null object
experiment_name    49 non-null object
dtypes: int64(3), object(7)
memory usage: 4.0+ KB
